Below is a Python code example using a Fully Convolutional Network (FCN) for semantic segmentation with 4x4 pixel chips. This example uses PyTorch for the FCN implementation and handles input images that are not perfectly divisible by the chip size by padding the image.

In [ ]:
import pandas as pd
import rasterio
import geopandas as gpd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Function to extract pixel chips


def extract_pixel_chips(gdf, raster, chip_size=4):
    half_chip = chip_size // 2
    values = []
    for point in gdf.geometry:
        row, col = raster.index(point.x, point.y)
        pixel_values = raster.read(
        )[:, row-half_chip:row+half_chip, col-half_chip:col+half_chip]
        values.append(pixel_values)
    return np.array(values)

# Fully Convolutional Network (FCN) for semantic segmentation


class FCN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(FCN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        return x

# Function to pad the input image so it's dimensions are divisible by the chip size


def pad_image(image, chip_size=4):
    rows, cols = image.shape[1], image.shape[2]
    pad_rows = chip_size - (rows % chip_size) if rows % chip_size != 0 else 0
    pad_cols = chip_size - (cols % chip_size) if cols % chip_size != 0 else 0
    padded_image = np.pad(image, ((0, 0), (0, pad_rows),
                          (0, pad_cols)), mode='constant')
    return padded_image


# Read CSV and create GeoDataFrame
csv_file = 'jaguar_locations.csv'
df = pd.read_csv(csv_file)
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Read satellite image
satellite_image = 'satellite_image.tif'
with rasterio.open(satellite_image) as src:
    image = src.read()
    profile = src.profile

# Pad the image if its dimensions are not divisible by the chip size
image = pad_image(image, chip_size=4)

# Extract pixel chips and labels
X = extract_pixel_chips(gdf, src)
y = gdf['habitat'].values

# Train the FCN
# Note: For a complete example, you would need to split the data into training and testing sets,
# create a PyTorch dataset and dataloader, and train the FCN using the dataloader.
# This is just an outline of the process.

num_classes = len(np.unique(y))
input_channels = X.shape[1]
model = FCN(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# After training the model, perform semantic segmentation on the input image by applying
# the model on the entire padded image, and then crop the result back to the original image dimensions

model.eval()
with torch.no_grad():
    input_image = torch.tensor(image[np.newaxis], dtype=torch.float32)
    output = model(input_image)
    predictions = output.argmax(dim=1).numpy()[0]

# Crop the padded predictions back to the original image dimensions
original_rows, original_cols = profile["height"], profile["width"]
predictions_cropped = predictions[:original_rows, :original_cols]

# Save the classified image as a new GeoTIFF
output_file = 'classified_jaguar_habitat.tif'

output_profile = profile.copy()
output_profile.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw'
)

with rasterio.open(output_file, 'w', **output_profile) as dst:
    dst.write(predictions_cropped.astype(rasterio.uint8), 1)


This part of the code shows how to perform semantic segmentation on the entire (padded) input image using the trained FCN model. The resulting classified image is then cropped back to the original image dimensions and saved as a new GeoTIFF file. Note that the training step is not included in this code snippet; you would need to create a PyTorch dataset and dataloader, and train the FCN using the dataloader as mentioned in the comments of the previous code block.

In [ ]:
# the code for training the FCN model using PyTorch:

# Custom Dataset class
class JaguarHabitatDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.long)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Create PyTorch datasets and dataloaders for training and testing
train_dataset = JaguarHabitatDataset(X_train, y_train)
test_dataset = JaguarHabitatDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Training the FCN model
epochs = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print the average loss for this epoch
    print(
        f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")

# Evaluate the trained model on the test dataset
model.eval()
test_accuracy = 0
total_samples = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predictions = outputs.argmax(dim=1)
        correct = predictions.eq(labels).sum().item()
        test_accuracy += correct
        total_samples += labels.size(0)

test_accuracy /= total_samples
print(f"Test accuracy: {test_accuracy * 100:.2f}%")
